In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

def analyze_expansion_data(file_path):
    # --- 1. 读取数据 ---
    print("正在读取数据...")
    try:
        df = pd.read_excel(file_path)
    except FileNotFoundError:
        print(f"错误：找不到文件 {file_path}")
        return

    # --- 2. 数据清洗与预处理 ---
    print("正在清洗数据...")
    
    # 数值列填充0，防止计算报错
    numeric_cols = [
        '原有合同容量', '申请合同容量', '原有运行容量', '申请运行容量', '运行容量', '合同容量',
        '业扩变更前预计2月份日均电量', '业扩变更后预计2月份日均电量', '业扩变更后预计3月份日均电量'
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    # 日期列处理
    date_col = '预计送停电日期'
    df['送电日期_dt'] = pd.to_datetime(df[date_col], errors='coerce')
    
    # --- 3. 核心逻辑：计算“有效变动容量” ---
    print("正在计算有效变动容量...")
    
    def get_effective_capacity(row):
        b_type = str(row['业扩业务类型'])
        
        # 1. 高压新装：看合同容量
        if '新装' in b_type:
            return row['合同容量']
        
        # 2. 高压增容：优先看申请合同容量，如果是0，看申请运行容量
        elif '增容' in b_type:
            if row['申请合同容量'] > 0:
                return row['申请合同容量']
            else:
                return row['申请运行容量']
        
        # 3. 减容：看申请运行容量，或者用原有-现有
        elif '减容' in b_type:
            if row['申请运行容量'] > 0:
                return row['申请运行容量']
            else:
                diff = row['原有运行容量'] - row['运行容量']
                return diff if diff > 0 else 0
        
        return 0

    df['有效变动容量'] = df.apply(get_effective_capacity, axis=1)

    # --- 4. 核心逻辑：计算电量增量 (2月每周 & 3月) ---
    print("正在计算电量增量...")

    # 2月日均增量 = 2月变更后 - 2月变更前
    df['日均增量_2月'] = df['业扩变更后预计2月份日均电量'] - df['业扩变更前预计2月份日均电量']
    
    # 【核心修改点】3月日均增量 = 3月变更后 - 2月变更后
    df['日均增量_3月'] = df['业扩变更后预计3月份日均电量'] - df['业扩变更后预计2月份日均电量']

    # 定义2月的时间段 (2026年)
    weeks_config = [
        ('2月第一周', pd.Timestamp('2026-02-02'), pd.Timestamp('2026-02-08')),
        ('2月第二周', pd.Timestamp('2026-02-09'), pd.Timestamp('2026-02-15')),
        ('2月第三周', pd.Timestamp('2026-02-16'), pd.Timestamp('2026-02-22')),
        ('2月第四周', pd.Timestamp('2026-02-23'), pd.Timestamp('2026-02-28'))
    ]

    def calculate_kwh_increment(row, start_date, end_date, daily_inc):
        """计算某段时间内的电量增量"""
        if pd.isna(row['送电日期_dt']):
            return 0
        
        effect_date = row['送电日期_dt']
        
        # 如果送电日期在时间段结束之后，该时间段无增量
        if effect_date > end_date:
            return 0
        
        # 有效开始时间 = max(时间段开始, 送电日期)
        actual_start = max(start_date, effect_date)
        
        if actual_start > end_date:
            return 0
            
        days = (end_date - actual_start).days + 1
        return days * daily_inc

    # 计算2月每周增量
    for week_name, w_start, w_end in weeks_config:
        df[f'{week_name}增量'] = df.apply(
            lambda row: calculate_kwh_increment(row, w_start, w_end, row['日均增量_2月']), axis=1
        )

    # 计算3月全月增量
    mar_start = pd.Timestamp('2026-03-01')
    mar_end = pd.Timestamp('2026-03-31')
    
    # 注意：3月的增量计算逻辑
    # 如果送电日期在3月之前（比如2月已送电），则3月全月都产生这个“3月-2月”的差额增量
    # 如果送电日期在3月之间，则从送电日期开始产生增量
    df['3月总增量'] = df.apply(
        lambda row: calculate_kwh_increment(row, mar_start, mar_end, row['日均增量_3月']), axis=1
    )
    
    # 计算2月合计增量
    feb_cols = [w[0]+'增量' for w in weeks_config]
    df['2月合计增量'] = df[feb_cols].sum(axis=1)

    # --- 5. 聚合分析 ---
    print("正在生成分析报表...")

    agg_funcs_cap = {
        '有效变动容量': 'sum',
        '用户编号': 'count'
    }
    
    agg_funcs_power = {
        '2月第一周增量': 'sum',
        '2月第二周增量': 'sum',
        '2月第三周增量': 'sum',
        '2月第四周增量': 'sum',
        '2月合计增量': 'sum',
        '3月总增量': 'sum'
    }

    # 5.1 按 133行业 + 业务类型
    df_industry_cap = df.groupby(['133行业名称', '业扩业务类型']).agg(agg_funcs_cap).reset_index()
    df_industry_cap.rename(columns={'用户编号': '户数', '有效变动容量': '容量(kVA)'}, inplace=True)
    
    # 5.2 按 市场化属性 + 业务类型
    df_market_cap = df.groupby(['市场化属性分类名称', '业扩业务类型']).agg(agg_funcs_cap).reset_index()
    df_market_cap.rename(columns={'用户编号': '户数', '有效变动容量': '容量(kVA)'}, inplace=True)

    # 5.3 按 133行业 统计电量
    df_industry_power = df.groupby(['133行业名称']).agg(agg_funcs_power).reset_index()
    df_industry_power = df_industry_power.sort_values(by='2月合计增量', ascending=False)

    # 5.4 按 市场化属性 统计电量
    df_market_power = df.groupby(['市场化属性分类名称']).agg(agg_funcs_power).reset_index()
    
    # 5.5 Top 用户 (按2月增量绝对值排序)
    df['2月增量绝对值'] = df['2月合计增量'].abs()
    df_top_users = df.sort_values(by='2月增量绝对值', ascending=False).head(50)
    
    top_user_cols = [
        '地市', '用户名称', '133行业名称', '业扩业务类型', '有效变动容量', '预计送（停）电日期', 
        '业扩变更前预计2月份日均电量', '业扩变更后预计2月份日均电量', '业扩变更后预计3月份日均电量',
        '2月合计增量', '3月总增量'
    ]
    # 确保列存在再筛选，防止报错
    existing_cols = [c for c in top_user_cols if c in df.columns]
    df_top_users_export = df_top_users[existing_cols]

    # --- 6. 导出结果 ---
    output_path = os.path.join(os.path.dirname(file_path), '业扩摸排数据_分析结果.xlsx')
    print(f"正在保存结果至: {output_path}")
    
    with pd.ExcelWriter(output_path) as writer:
        df_industry_cap.to_excel(writer, sheet_name='行业_容量及户数', index=False)
        df_market_cap.to_excel(writer, sheet_name='市场化_容量及户数', index=False)
        df_industry_power.to_excel(writer, sheet_name='行业_电量增量预测', index=False)
        df_market_power.to_excel(writer, sheet_name='市场化_电量增量预测', index=False)
        df_top_users_export.to_excel(writer, sheet_name='TOP50增量用户', index=False)
        
        # 保存明细表用于核对
        detail_cols = [
            '用户名称', '133行业名称', '送电日期_dt', 
            '业扩变更前预计2月份日均电量', '业扩变更后预计2月份日均电量', '业扩变更后预计3月份日均电量',
            '日均增量_2月', '日均增量_3月', '2月合计增量', '3月总增量'
        ]
        df[detail_cols].to_excel(writer, sheet_name='计算明细核对', index=False)

    print("分析完成！")

# --- 执行 ---
if __name__ == '__main__':
    file_path = r"E:\A智网\业扩分析\26年2月分析\业扩摸排数据.xlsx"
    analyze_expansion_data(file_path)

正在读取数据...
正在清洗数据...


KeyError: '预计送\\停电日期'